In [1]:
sc._jsc.hadoopConfiguration().set(
  "fs.azure.account.key.sankalpstoragesapient.blob.core.windows.net",
  "QTxaNp0ti9TOLjVGIZy3Ftx8xGoq/thTy/WdzDuBzgoCwbqdg5jIqQt6OqnE5DUrF0g01Zr5jrRtEO0WVAYrxg==")
 
spark.conf.set(
  "fs.azure.account.key.sankalpstoragesapient.blob.core.windows.net",
  "QTxaNp0ti9TOLjVGIZy3Ftx8xGoq/thTy/WdzDuBzgoCwbqdg5jIqQt6OqnE5DUrF0g01Zr5jrRtEO0WVAYrxg==")
 

In [2]:
#import required libraries
import numpy as np
import matplotlib.pyplot as plt
import pyspark
import re
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *
from datetime import datetime
import os
from IPython.display import Image
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier

In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('page_imp_italy').getOrCreate()

In [4]:
#load dataset into spark dataframe
df = spark.read.csv("wasbs://hashed-italy@sankalpstoragesapient.blob.core.windows.net/seven/2018-09-10/*", header=True,sep='\t')

In [5]:
#load the page mapping file
page_mapping_uri=spark.read.csv("wasbs://hashed-italy@sankalpstoragesapient.blob.core.windows.net/pagename_category.csv",header=True)

In [6]:
#create page mapping dictionary 
dict_pagename_mapping = {row['PageName']:row['Category'] for row in page_mapping_uri.select('PageName','Category').collect()}


In [7]:
def udf_wrapper(returntype):
    def udf_func(func):
        return udf(func, returnType=returntype)
    return udf_func

  
@udf_wrapper(StringType())
def get_page_category(page_type):
    '''
    based on the page_type return page category
    '''
    if page_type is None:
        return None
    if page_type == 'null':
        return None
    if page_type in dict_pagename_mapping.keys():
        return dict_pagename_mapping[page_type]
    else:
        return None

In [8]:
#create an additional column to add page category in dataframe
df=df.withColumn('page_category', get_page_category(col('post_pagename')))

In [9]:
#page category count
page_category_count=df.groupBy('page_category').count().orderBy('count',ascending=False)

In [10]:
#save the category count back to storage 
page_category_count.coalesce(1).write.format("csv").option("header","true").save("wasbs://hashed-italy@sankalpstoragesapient.blob.core.windows.net/pagecategory_count.csv")

In [11]:
#number of records in the data 
df.count()

Out[ 17 ]: 579866825

In [12]:
#number of unique visitors
df.select('masked_mcvisid').distinct().count()

Out[ 18 ]: 3387202

In [13]:
#number of actual test drive booking 
df.filter(col('page_category')=='test_drive').select('masked_mcvisid').distinct().count()

Out[ 22 ]: 13056

In [14]:
#number of actual test drive booking 
df.filter(col('page_category')=='your_vehicle').select('masked_mcvisid').distinct().count()

Out[ 23 ]: 398246

In [15]:
#number of records for each user 
df_agg=df.groupBy('masked_mcvisid').count().orderBy('count',ascending=False)

In [16]:
#filter outliers with high or less number of activities 
df_filter=df_agg.filter(df_agg['count'] < 10000).filter(df_agg['count'] > 2)

In [17]:
#genuine visitor count
df_filter.count()

Out[ 26 ]: 2933930

In [18]:
#filter genuine visitors only for propensity model
df_processed = df.join(df_filter, on='masked_mcvisid', how='inner')

In [19]:
#data cleaning 
df_processed = df_processed.filter(col('hit_source') == '1') \
                    .filter(col('exclude_hit') == '0') \
                    .filter(col('page_category') != '')

In [20]:
# sessionization
TIME_OUT = 30 # 30 minutes of timeout


@udf_wrapper(IntegerType())
def get_event_boundary(time_diff):
    
    if(time_diff is None):
      return 0
    else:
        if(time_diff > TIME_OUT):
            return 1
        else:
            return 0

@udf_wrapper(FloatType())
def set_default_page_time(time_spent):
    '''
    based on the time spent, if if
    '''
    if(time_spent is None):
        return 0.08 # 5 seconds 
    else:
        return time_spent

In [21]:
def conversion(page_category):
        if page_category == 'test_drive':
            return 1
        else : 
            return 0
    
conversion_udf = udf(conversion, IntegerType())

In [22]:
ts_pattern_1 = 'yyyy-MM-dd HH:mm:ss'
date_pattern_1 = 'yyyy-MM-dd'

df_processed = df_processed \
                     .withColumn('date_time_ts',unix_timestamp(col('date_time'), ts_pattern_1).cast('timestamp'))\
                     .withColumn('conversion_status', conversion_udf('page_category'))

In [23]:
    
# Sessionization
w0 = Window.partitionBy(col('masked_mcvisid'))

w1 = Window.partitionBy(col('masked_mcvisid')).orderBy(col('date_time_ts'))

w2 = Window.partitionBy(col('masked_mcvisid'),col('session_id')).orderBy(col('date_time_ts').desc())

w3 = Window.partitionBy(col('masked_mcvisid'),col('session_id')).orderBy(col('date_time_ts'))

df_processed  = df_processed \
                   .withColumn('is_converted', max(col('conversion_status')).over(w0)) \
                   .withColumn('prev', lag(col('date_time_ts'),1).over(w1)) \
                   .withColumn('time_diff',((col('date_time_ts').cast('long') - col('prev').cast('long'))/60.0)) \
                   .withColumn('new_event_boundary',get_event_boundary(col('time_diff'))) \
                   .withColumn('session_id', sum(col('new_event_boundary')).over(w1)) \
                   .drop('prev','time_diff','new_event_boundary')

                    
# hit order
df_processed = df_processed.withColumn('hit_rank_reversed', dense_rank().over(w2))



#Time Spent                      
df_processed = df_processed\
                    .withColumn('next', lead(col('date_time_ts'),1).over(w3)) \
                    .withColumn('time_diff',((col('next').cast('long') - col('date_time_ts').cast('long'))/60.0)) \
                    .withColumn('time_spent_in_mins',set_default_page_time(col('time_diff'))) \
                    .drop('next','time_diff')


In [24]:
df_final=df_processed.select('masked_mcvisid','page_category','time_spent_in_mins','is_converted')

In [25]:
#saving dataframe as cacheTable
df_final.createOrReplaceTempView('df_hits_view')
spark.catalog.cacheTable('df_hits_view')
df_hits = spark.table("df_hits_view")

In [26]:
df_hits.select('masked_mcvisid','is_converted').distinct().count()

Out[ 35 ]: 2665618

In [27]:
data_final=df_hits.filter((df_hits['is_converted']=='1') |  (df_hits['is_converted']=='0'))

In [28]:
#Ensuring no null values for pagename
data_final=data_final.filter(data_final['page_category'] !='null')

In [29]:
#unique visitors and conversion status dataframe to be used later
pos_users=data_final.select(['masked_mcvisid','is_converted']).distinct()

In [30]:
#creating list of unique pages 
page_list=data_final.select('page_category').distinct().collect()

#creating an list of equal size of distinct pages to map and replace with pagenames in dataset
page_id=list(range(1,(len(page_list)+1)))

#converting the page id list into strings to be used as columns later
page_id_str=list(map(str, page_id))


#creating the dictionary with page id mapped to pagename
page_dict = {x[0]: y for x,y in zip(page_list, page_id_str)}

In [31]:
#creating a udf to map the url pages to url ids
def page_map(page):
    return page_dict[page]

#implementing the udf and adding a new column (url status) to the datset
page_udf = udf(page_map, StringType())
finaldata= data_final.withColumn('page_status',page_udf(col('page_category')))

aggregations = []
aggregations.append(count(col('page_status')).alias('page_count'))

#creating a new dataset with aggregated values to be used in machine learning model
df_test=finaldata \
        .groupBy(['masked_mcvisid','page_status']) \
        .agg(*aggregations)

In [32]:
#creating a processed dataset with each url id as a new column for each unique visitor id
data_reshaped=df_test.groupBy('masked_mcvisid').pivot('page_status').sum('page_count').fillna(0)

In [33]:
data_reshaped.show(5)

+--------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
 masked_mcvisid| 1| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 2| 20| 21| 22| 23| 24| 25| 26| 27| 28| 29| 3| 30| 31| 32| 33| 4| 5| 6| 7| 8| 9|
+--------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
00c2a3affcdef3c4b...| 0| 1| 0| 1| 0| 0| 7| 0| 0| 0| 0| 0| 0| 2| 1| 3| 0| 0| 0| 3| 2| 0| 0| 0| 0| 0| 0| 1| 1| 0| 0| 0| 0|
0245e0e625e057a6b...| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
0540b269e6c5f7b07...| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 2| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
0b308f5d521bd4e9f...| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 28| 0| 0| 0| 5| 0| 2| 0| 0| 0| 0| 0| 0| 0| 0| 1| 9| 0|
0b32085d8746d0538...| 0| 0| 0| 0| 0| 0| 2| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+--------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
only showing top 5 rows

In [34]:
final_model_data = data_reshaped.join(pos_users, on='masked_mcvisid', how='inner')

In [35]:
page_dict 

Out[ 43 ]: 
{'passengercars': '15',
 'your_vehicle': '20',
 'print_layer': '11',
 'eco': '18',
 'interior': '22',
 'errors': '25',
 'chat': '13',
 'not_set': '16',
 'owner': '33',
 'feasibility_abort': '29',
 'connectivity': '10',
 'finance': '4',
 'business_cars': '9',
 'fleet_and_business': '17',
 'item_info': '7',
 'test_drive': '6',
 'others': '14',
 'configurator': '23',
 'moterization': '27',
 'safety': '5',
 'dealer_locator': '19',
 'home': '28',
 'design': '30',
 'user_cars': '21',
 'feasibility_conflict': '8',
 'vehicle_search': '24',
 'overview': '1',
 'contact-us': '3',
 'connect_store': '2',
 'exterior': '12',
 'online_code': '26',
 'offers': '32',
 'models': '31'}

In [36]:
test_drive_dict_value=page_dict['test_drive']

In [37]:
excluded_columns=['masked_mcvisid','is_converted',test_drive_dict_value]

In [38]:
model_inputs=[col for col in final_model_data.columns if col not in excluded_columns]

In [39]:
#Machine Learning Model

def page_imp_model(model_inputs,data):
    
    
    #creating the assemblerfor input variables 
    assembler=VectorAssembler(inputCols=model_inputs,outputCol='features')


    #creating dense vector represntation of input variables
    output=assembler.transform(final_model_data)

    #declaring the input dense vector and output variable
    data=output.select('features','is_converted')

    #build and train the ML model
    rfc=RandomForestClassifier(labelCol='is_converted',featuresCol='features')


    #fit the model on training data 
    rf_model=rfc.fit(data)
    
    #map the page ids back to pagenames
    feats={}
    for feature,importance in zip(model_inputs,rf_model.featureImportances):
        feats[feature]=importance


    results=sorted(feats.items(),key=lambda x:x[1],reverse=True)

    page_dict_rev = {y:x for x,y in page_dict.items()}

    #create dataframe with page and importance
    final_results_page=[(page_dict_rev[key],str(imp)) for key,imp in results]
    final_results_page_df=spark.createDataFrame(final_results_page,['page_category','importance'])
    return final_results_page_df



In [40]:
page_results=page_imp_model(model_inputs,final_model_data)

In [41]:
page_results.show(100,False)

+--------------------+-----------------+
page_category |importance |
+--------------------+-----------------+
passengercars |0.140344277547 |
others |0.134461458003 |
contact-us |0.120489044054 |
owner |0.078220592708 |
design |0.0639875369059 |
finance |0.0622014313542 |
fleet_and_business |0.0608427498631 |
offers |0.0481786250678 |
dealer_locator |0.0468594042156 |
moterization |0.0452552094626 |
home |0.0331879029136 |
configurator |0.0326866936709 |
eco |0.0195279238448 |
user_cars |0.0152679903111 |
chat |0.0135336147488 |
vehicle_search |0.0133952065773 |
item_info |0.0125540020934 |
feasibility_conflict|0.0121365155868 |
business_cars |0.00755847375169 |
interior |0.00735244859513 |
feasibility_abort |0.00620223593645 |
your_vehicle |0.00604677805739 |
exterior |0.00588333551497 |
connectivity |0.00482261625895 |
online_code |0.00312540319989 |
safety |0.0026391707101 |
print_layer |0.00207427879779 |
models |0.000621330885209|
connect_store |0.000317604781969|
not_set |0.000226144583265|
overview |0.0 |
errors |0.0 |
+--------------------+-----------------+

In [42]:
page_counts=data_final.groupBy('page_category').count().orderBy('count',ascending=True)

In [43]:
page_time_spent=data_final.groupBy('page_category').agg({"time_spent_in_mins": "sum"}).alias("total_time_spent")

In [44]:
df_merge_1 = page_results.join(page_time_spent, on='page_category', how='inner')

In [45]:
df_merge=df_merge_1.join(page_counts ,on='page_category', how='inner')

In [46]:
df_merge.show(50,False)

+--------------------+-----------------+-----------------------+--------+
page_category |importance |sum(time_spent_in_mins)|count |
+--------------------+-----------------+-----------------------+--------+
overview |0.0 |1.776666671037674 |5 |
connect_store |0.000317604781969|45989.233353929594 |47442 |
contact-us |0.120489044054 |60607.336636586115 |64052 |
finance |0.0622014313542 |302732.3098520059 |228616 |
safety |0.0026391707101 |19746.416775638238 |71325 |
item_info |0.0125540020934 |1002520.9413638525 |1507814 |
feasibility_conflict|0.0121365155868 |1417049.4776352998 |7910255 |
business_cars |0.00755847375169 |51127.10333178379 |63401 |
connectivity |0.00482261625895 |14846.933481987566 |96447 |
print_layer |0.00207427879779 |78259.30329265259 |44555 |
exterior |0.00588333551497 |37011.913592431694 |147230 |
chat |0.0135336147488 |18260.350013582036 |16466 |
others |0.134461458003 |154387.50322023593 |163101 |
passengercars |0.140344277547 |5070475.159153907 |4352893 |
not_set |0.000226144583265|334.39333465509117 |1579 |
fleet_and_business |0.0608427498631 |67761.06667587347 |58185 |
eco |0.0195279238448 |68855.40662708133 |77677 |
dealer_locator |0.0468594042156 |45252.033313605934 |43354 |
user_cars |0.0152679903111 |347415.66301743314 |305136 |
your_vehicle |0.00604677805739 |1191983.5327074062 |944205 |
interior |0.00735244859513 |39330.01353548281 |140908 |
configurator |0.0326866936709 |9789411.127506828 |20625394|
vehicle_search |0.0133952065773 |10587.023356415331 |15898 |
errors |0.0 |184.0433335788548 |198 |
online_code |0.00312540319989 |4425.023350354284 |9891 |
moterization |0.0452552094626 |2695789.7116467822 |4906744 |
home |0.0331879029136 |99092.74374614283 |304187 |
design |0.0639875369059 |13988.110007168725 |15085 |
feasibility_abort |0.00620223593645 |299789.76903378405 |1123087 |
models |0.000621330885209|2447.0499939210713 |3332 |
offers |0.0481786250678 |269436.3398547247 |187217 |
owner |0.078220592708 |341250.6263089664 |324385 |
+--------------------+-----------------+-----------------------+--------+

In [47]:
df_merge.orderBy("importance", ascending=False).show(50,False)

+--------------------+-----------------+-----------------------+--------+
page_category |importance |sum(time_spent_in_mins)|count |
+--------------------+-----------------+-----------------------+--------+
passengercars |0.140344277547 |5070475.159153907 |4352893 |
others |0.134461458003 |154387.50322023593 |163101 |
contact-us |0.120489044054 |60607.336636586115 |64052 |
owner |0.078220592708 |341250.6263089664 |324385 |
design |0.0639875369059 |13988.110007168725 |15085 |
finance |0.0622014313542 |302732.3098520059 |228616 |
fleet_and_business |0.0608427498631 |67761.06667587347 |58185 |
offers |0.0481786250678 |269436.3398547247 |187217 |
dealer_locator |0.0468594042156 |45252.033313605934 |43354 |
moterization |0.0452552094626 |2695789.7116467822 |4906744 |
home |0.0331879029136 |99092.74374614283 |304187 |
configurator |0.0326866936709 |9789411.127506828 |20625394|
eco |0.0195279238448 |68855.40662708133 |77677 |
user_cars |0.0152679903111 |347415.66301743314 |305136 |
chat |0.0135336147488 |18260.350013582036 |16466 |
vehicle_search |0.0133952065773 |10587.023356415331 |15898 |
item_info |0.0125540020934 |1002520.9413638525 |1507814 |
feasibility_conflict|0.0121365155868 |1417049.4776352998 |7910255 |
business_cars |0.00755847375169 |51127.10333178379 |63401 |
interior |0.00735244859513 |39330.01353548281 |140908 |
feasibility_abort |0.00620223593645 |299789.76903378405 |1123087 |
your_vehicle |0.00604677805739 |1191983.5327074062 |944205 |
exterior |0.00588333551497 |37011.913592431694 |147230 |
connectivity |0.00482261625895 |14846.933481987566 |96447 |
online_code |0.00312540319989 |4425.023350354284 |9891 |
safety |0.0026391707101 |19746.416775638238 |71325 |
print_layer |0.00207427879779 |78259.30329265259 |44555 |
models |0.000621330885209|2447.0499939210713 |3332 |
connect_store |0.000317604781969|45989.233353929594 |47442 |
not_set |0.000226144583265|334.39333465509117 |1579 |
overview |0.0 |1.776666671037674 |5 |
errors |0.0 |184.0433335788548 |198 |
+--------------------+-----------------+-----------------------+--------+

In [48]:
#save final results back to storage 
df_merge.coalesce(1).write.format("csv").option("header","true").save("wasbs://hashed-italy@sankalpstoragesapient.blob.core.windows.net/page_importance")